<a href="https://colab.research.google.com/github/markmaci/ml-ai-project/blob/cnn_branch/ML_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Convolutional Neural Networks

## 1.1. Initial CNN Model

In [ ]:
!pip install utils

  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13906 sha256=76b5274fb0440d2cf8f1750dfcd679a81ba26ed53e37bab5de8890dfc0b17e86
  Stored in directory: /root/.cache/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils


In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.initializers import Constant


In [ ]:
!pip install datasets
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("sentiment140")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/498 [00:00<?, ? examples/s]

In [ ]:
df = pd.DataFrame(dataset['train'])

In [ ]:
# Rename 'sentiment' to 'target'
df = df.rename(columns={"sentiment": "target"})

# Drop rows with NaN values in 'target' and 'text'
df.dropna(subset=['target', 'text'], inplace=True)


In [ ]:
# convert to binary classification
df['target'] = df['target'].apply(lambda x: 1 if x == 4 else 0)

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

In [ ]:
max_words = 10000
max_len = 100

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=0.2, random_state=42)

In [ ]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout

embedding_dim = 100

model = Sequential([
    Embedding(max_words, embedding_dim, input_length=max_len),
    Conv1D(128, 5, activation='relu'),
    MaxPooling1D(pool_size=5),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1000000   
                                                                 
 conv1d (Conv1D)             (None, 96, 128)           64128     
                                                                 
 max_pooling1d (MaxPooling1  (None, 19, 128)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 15, 128)           82048     
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 128)               1

In [ ]:
history = model.fit(X_train_pad, y_train, epochs=1, batch_size=32, validation_split=0.2)

32000/32000 [==============================] - 1754s 55ms/step - loss: 0.4682 - accuracy: 0.7795 - val_loss: 0.4437 - val_accuracy: 0.7935


In [ ]:
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f'Test Accuracy: {accuracy}')

10000/10000 [==============================] - 136s 14ms/step - loss: 0.4429 - accuracy: 0.7925
Test Accuracy: 0.7925406098365784


**FROM HERE**

## 1.2. Enhanced CNN Model

### 1.2.1 GloVe embeddings

In [ ]:
# Download the GloVe embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

 Load GloVe Embeddings into a Matrix:

Pre-trained GloVe embeddings capture semantic relationships between words based on vast amounts of text data, providing a richer starting point for the model compared to random initialization.

In [ ]:
embedding_dim = 100
embedding_index = {}

# Make sure the 'glove.6B.100d.txt' file is in your working directory
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
max_len = 150  # Increased sequence length!!

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=0.2, random_state=42)

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)


In the enhanced model, several significant improvements have been made:

- Pre-trained GloVe embeddings have been integrated for richer word representations.
- An additional Conv1D layer has been added for capturing more complex features.
MaxPooling1D layers follow each convolutional layer to downsample the feature maps and reduce dimensionality, which helps in capturing the most prominent features
- Dropout rates have been adjusted and additional dropout layers have been added for better regularization. Dropout rates are set to 0.2 after the first two Conv1D layers and 0.5 before the output layer. This regularization technique helps in reducing overfitting.
- Added a Dense layer with 128 units, activated by ReLU, followed by a dropout layer with a dropout rate of 0.5. This fully connected layer helps in combining features extracted by convolutional layers before making the final prediction.
The output layer is a Dense layer with a single unit activated by sigmoid, suitable for binary classification.
- Early stopping and learning rate reduction callbacks have been implemented to optimize training and improve generalization.




In [ ]:
# Enhanced Model
model1 = Sequential([
    Embedding(max_words, embedding_dim, embeddings_initializer=Constant(embedding_matrix), input_length=max_len, trainable=False),
    Conv1D(128, 5, activation='relu'),
    MaxPooling1D(pool_size=5),
    Dropout(0.2),
    Conv1D(128, 5, activation='relu'),
    MaxPooling1D(pool_size=5),
    Dropout(0.2),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model1.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 150, 100)          1000000   
                                                                 
 conv1d_8 (Conv1D)           (None, 146, 128)          64128     
                                                                 
 max_pooling1d_5 (MaxPoolin  (None, 29, 128)           0         
 g1D)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 29, 128)           0         
                                                                 
 conv1d_9 (Conv1D)           (None, 25, 128)           82048     
                                                                 
 max_pooling1d_6 (MaxPoolin  (None, 5, 128)            0         
 g1D)                                                 

Initially, training the enhanced model using 1 epoch with callbacks:

(around 45 min to run)

In [ ]:
# Callbacks for Early Stopping and Learning Rate Reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

# Training the Model with Callbacks
history = model1.fit(X_train_pad, y_train, epochs=1, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr])


32000/32000 [==============================] - 2703s 84ms/step - loss: 0.5371 - accuracy: 0.7305 - val_loss: 0.5032 - val_accuracy: 0.7530 - lr: 0.0010


In [ ]:
# Evaluating the Model
loss, accuracy = model1.evaluate(X_test_pad, y_test)
print(f'Test Accuracy: {accuracy}')


10000/10000 [==============================] - 312s 31ms/step - loss: 0.5023 - accuracy: 0.7523
Test Accuracy: 0.7523468732833862


Training the enhanced model using 10 epochs:

In [ ]:
# Callbacks for Early Stopping and Learning Rate Reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

# Continue training for more epochs
history = model1.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr])


Epoch 1/10
32000/32000 [==============================] - 2587s 81ms/step - loss: 0.5123 - accuracy: 0.7492 - val_loss: 0.4946 - val_accuracy: 0.7602 - lr: 0.0010
Epoch 2/10
32000/32000 [==============================] - 2488s 78ms/step - loss: 0.5032 - accuracy: 0.7568 - val_loss: 0.4944 - val_accuracy: 0.7607 - lr: 0.0010
Epoch 3/10
32000/32000 [==============================] - 2442s 76ms/step - loss: 0.4982 - accuracy: 0.7597 - val_loss: 0.4899 - val_accuracy: 0.7654 - lr: 0.0010
Epoch 4/10
32000/32000 [==============================] - 2429s 76ms/step - loss: 0.4940 - accuracy: 0.7627 - val_loss: 0.4840 - val_accuracy: 0.7659 - lr: 0.0010
Epoch 5/10
32000/32000 [==============================] - 2399s 75ms/step - loss: 0.4917 - accuracy: 0.7645 - val_loss: 0.4851 - val_accuracy: 0.7672 - lr: 0.0010
Epoch 6/10
27941/32000 [=========================>....] - ETA: 4:58 - loss: 0.4884 - accuracy: 0.7666

Training the enhanced model using 50 epochs:

In [ ]:
# Callbacks for Early Stopping and Learning Rate Reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

# Continue training for more epochs
history = model1.fit(X_train_pad, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

In [ ]:
# Evaluate the model after extended training
final_loss, final_accuracy = model1.evaluate(X_test_pad, y_test)
print(f'Final Test Accuracy after additional training: {final_accuracy}')

In practice, it's often observed that:

- Initial Epochs: A simpler model might perform better in the initial few epochs because it is easier to train and converges quickly on simpler patterns.
- Subsequent Epochs: As training progresses, the enhanced model typically surpasses the simpler model due to its greater capacity to learn complex patterns and its better regularization.


The enhanced model should achieve better accuracy with 10 epochs compared to initial model using the same number of epochs.

### 1.2.2. Word2Vec embeddings

In [ ]:
!pip install gensim


(15 min to run):

In [ ]:
import gensim.downloader as api
from gensim.models import KeyedVectors

# Download the Word2Vec model (this might take some time)
word2vec_model = api.load('word2vec-google-news-300')


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
import numpy as np

embedding_dim = 300  # Word2Vec embeddings have 300 dimensions
word_index = tokenizer.word_index
max_words = len(word_index) + 1

# Initialize the embedding matrix
embedding_matrix = np.zeros((max_words, embedding_dim))

# Fill the embedding matrix with Word2Vec embeddings
for word, i in word_index.items():
    if i < max_words:
        try:
            embedding_vector = word2vec_model[word]
            embedding_matrix[i] = embedding_vector
        except KeyError:
            # Words not found in the embedding index will be all zeros
            continue


In [ ]:
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import Constant

# Define the enhanced model with Word2Vec embeddings
model1 = Sequential([
    Embedding(max_words, embedding_dim, embeddings_initializer=Constant(embedding_matrix), input_length=max_len, trainable=False),
    Conv1D(128, 5, activation='relu'),
    MaxPooling1D(pool_size=5),
    Dropout(0.2),
    Conv1D(128, 5, activation='relu'),
    MaxPooling1D(pool_size=5),
    Dropout(0.2),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model1.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 300)          178454700 
                                                                 
 conv1d (Conv1D)             (None, 146, 128)          192128    
                                                                 
 max_pooling1d (MaxPooling1  (None, 29, 128)           0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 29, 128)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 25, 128)           82048     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 5, 128)            0         
 g1D)                                                   

In [ ]:
# Callbacks for Early Stopping and Learning Rate Reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

# Train the enhanced model
history = model1.fit(X_train_pad, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

# Evaluate the model
final_loss, final_accuracy = model1.evaluate(X_test_pad, y_test)
print(f'Enhanced Model with Word2Vec Test Accuracy: {final_accuracy}')


## 1.3. RNN Model

In [ ]:
from tensorflow.keras.layers import LSTM, Bidirectional, SpatialDropout1D
from tensorflow.keras.models import Sequential

# Define the RNN model
model2 = Sequential([
    Embedding(max_words, embedding_dim, embeddings_initializer=Constant(embedding_matrix), input_length=max_len, trainable=True),
    SpatialDropout1D(0.2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model2.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 300)          178454700 
                                                                 
 spatial_dropout1d (Spatial  (None, 150, 300)          0         
 Dropout1D)                                                      
                                                                 
 bidirectional (Bidirection  (None, 200)               320800    
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               25728     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1

Training for 1 epoch using Word2Vec embeddings:

In [ ]:
# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

# Train the RNN model
history_rnn = model2.fit(X_train_pad, y_train, epochs=1, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

# Evaluate the RNN model
final_loss_rnn, final_accuracy_rnn = model2.evaluate(X_test_pad, y_test)
print(f'RNN Model Test Accuracy: {final_accuracy_rnn}')

## 1.4. Final Ensemble Model (CNN & RNN Combined)

In [ ]:
import numpy as np

# Get predictions from both models
predictions_cnn = model1.predict(X_test_pad)
predictions_rnn = model2.predict(X_test_pad)

# Average the predictions
ensemble_predictions = (predictions_cnn + predictions_rnn) / 2

# Convert probabilities to binary outcomes (0 or 1)
ensemble_predictions_binary = np.where(ensemble_predictions > 0.5, 1, 0)

# Calculate the accuracy of the ensemble model
ensemble_accuracy = np.mean(ensemble_predictions_binary == y_test.values)
print(f'Ensemble Model Test Accuracy: {ensemble_accuracy}')
